In [17]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM
from sklearn import preprocessing

In [25]:
bok = pd.read_csv('../../../data/bokhoven.csv')

In [26]:
bok.drop(['Unnamed: 0', 'datumBeginMeting'], axis=1, inplace=True) #don't need these

In [27]:
#transform to timestamp and add features
def AddColumns(df):
    df['Begin'] = pd.to_datetime(df['Begin'])
    df['Date'] = pd.DatetimeIndex(df.Begin).normalize()
    df['Weekday'] = df['Begin'].dt.day_name()
    #df['Hour'] = df['Begin'].dt.hour
    df['TimeOfDay'] = df['Begin'].dt.time
    #df['Year'] = df['Begin'].dt.year
    df['Month'] = df['Begin'].dt.month_name()
    #df['Day'] = df['Begin'].dt.day
    return df

In [32]:
new_bok = AddColumns(bok)
new_bok[63:70]

,hstWaarde,Bokhoven,Begin,cumsum_previous_15,rain_-15_class,Date,Weekday,TimeOfDay,Month
63,260.0,0.0,2018-01-03 15:00:00,0.0,0,2018-01-03,Wednesday,15:00:00,January
64,0.0,0.0,2018-01-03 16:00:00,0.0,0,2018-01-03,Wednesday,16:00:00,January
65,0.0,0.0,2018-01-03 17:00:00,0.0,0,2018-01-03,Wednesday,17:00:00,January
66,250.0,0.0,2018-01-03 18:00:00,0.0,0,2018-01-03,Wednesday,18:00:00,January
67,300.0,0.0,2018-01-03 19:00:00,0.0,0,2018-01-03,Wednesday,19:00:00,January
68,0.0,0.0,2018-01-03 20:00:00,0.0,0,2018-01-03,Wednesday,20:00:00,January
69,220.0,0.0,2018-01-03 21:00:00,0.0,0,2018-01-03,Wednesday,21:00:00,January


In [45]:
def preprocess_df(df):
    for col in df.columns:  # go through all of the columns
        if col == "hstWaarde":  # normalize the flow column 
            df[col] = (df[col]-df[col].min())/(df[col].max()- df[col].min()) #scale between 0 and 1 using min-max normalization
    df.dropna(inplace=True)  # cleanup again... jic. Those nasty NaNs love to creep in.
    return df

In [46]:
cleaned_data = new_bok[new_bok['rain_-15_class'] == 0]
preproc = preprocess_df(cleaned_data)

C:\Users\maren\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
C:\Users\maren\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [47]:
preproc.describe()

,hstWaarde,Bokhoven,cumsum_previous_15,rain_-15_class
count,9351.000000,9351.000000,9351.000000,9351.0
mean,0.122008,0.000081,0.000759,0.0
std,0.194193,0.001609,0.004716,0.0
min,0.000000,0.000000,0.000000,0.0
25%,0.000000,0.000000,0.000000,0.0
50%,0.050000,0.000000,0.000000,0.0
75%,0.183333,0.000000,0.000000,0.0
max,1.000000,0.048400,0.049100,0.0
